In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('dups_telefonos.csv')

In [3]:
df.head()

,Unnamed: 0,IDMOROSO,TIPO_DOC,NRO_DOC,CUIT_CUIL,TIPO,SUBTIPO,DATO,PROPIETARIO_DATO,PROVEEDOR_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,SCORE_DATO
0,1823,BBVA12063032,DNI,12063032,NaN,TEL.FIJO,PERSONAL,2293423790,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4
1,3123,BBVA11431130,DNI,11431130,NaN,TEL.FIJO,PERSONAL,2994487723,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4
2,3192,BBVA22000834,DNI,22000834,NaN,TEL.FIJO,PERSONAL,4290892318,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4
3,3197,BBVA22264120,DNI,22264120,NaN,TEL.FIJO,PERSONAL,3813073596,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4
4,3632,BBVA16805508,DNI,16805508,NaN,TEL.FIJO,PERSONAL,3462431099,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4


In [4]:
df.shape

(14316, 15)

In [5]:
df = df.drop(columns=['Unnamed: 0'])

In [6]:
#df = df.drop_duplicates(subset=['IDMOROSO','TIPO_DOC','NRO_DOC','CUIT_CUIL','TIPO','SUBTIPO','DATO','PROPIETARIO_DATO'
#                               ,'PROVEEDOR_DATO','FECHA_INGRESO_DATO','VERIFICADO','FECHA_VERIFICACION'
#                               ,'OPERADOR_VERIFICACION','SCORE_DATO'])

In [7]:
df = df.drop_duplicates()

In [8]:
df.shape

(14316, 14)

In [9]:
#df.groupby(['IDMOROSO', 'PROVEEDOR_DATO', 'DATO']).count()

## Dato a str + strip()

In [10]:
df ['DATO'] = df['DATO'].astype(str)

In [11]:
df['DATO'] = df['DATO'].str.strip()

## Verificados

Si verificado es false, pasamos a NaN la fecha de verificación

In [12]:
df.loc[df['VERIFICADO'] == False, 'FECHA_VERIFICACION'] = np.nan

In [13]:
df.tail()

,IDMOROSO,TIPO_DOC,NRO_DOC,CUIT_CUIL,TIPO,SUBTIPO,DATO,PROPIETARIO_DATO,PROVEEDOR_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,SCORE_DATO
14311,DNI 18485359,DNI,18485359,NaN,TEL.FIJO,PERSONAL,3414579001,Propio,AT,2021-01-14 16:03:30.252685,False,NaN,NaN,4
14312,DNI 18536029,DNI,18536029,NaN,TEL.FIJO,PERSONAL,294155532704,Propio,AT,2021-01-14 16:03:30.252685,False,NaN,NaN,4
14313,DNI 11655157,DNI,11655157,NaN,TEL.FIJO,PERSONAL,3464424817,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4
14314,DNI 23855748,DNI,23855748,NaN,TEL.FIJO,PERSONAL,3874227654,Propio,AT,2021-01-14 16:03:30.252685,False,NaN,NaN,4
14315,RMS376635547651009,CUIT,RMS376635547651009,NaN,TEL.FIJO,PERSONAL,47420876,Propio,AT,2021-01-14 16:03:30.252685,False,NaN,NaN,4


# Subset: Duplicados

Me quedo con los todos los que tienen duplicados la combinatoria IDMOROSO, DATO y PROVEEDOR DATO, y reviso qué pasa en cada caso

In [14]:
duplicates_subset = df[df.duplicated(subset=['IDMOROSO','DATO','PROVEEDOR_DATO'],keep=False)]

In [15]:
duplicates_subset.head()

,IDMOROSO,TIPO_DOC,NRO_DOC,CUIT_CUIL,TIPO,SUBTIPO,DATO,PROPIETARIO_DATO,PROVEEDOR_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,SCORE_DATO
0,BBVA12063032,DNI,12063032,NaN,TEL.FIJO,PERSONAL,2293423790,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4
1,BBVA11431130,DNI,11431130,NaN,TEL.FIJO,PERSONAL,2994487723,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4
2,BBVA22000834,DNI,22000834,NaN,TEL.FIJO,PERSONAL,4290892318,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4
3,BBVA22264120,DNI,22264120,NaN,TEL.FIJO,PERSONAL,3813073596,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4
4,BBVA16805508,DNI,16805508,NaN,TEL.FIJO,PERSONAL,3462431099,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4


In [16]:
duplicates_subset.shape

(14316, 14)

# 1. Puntajes

Asignamos puntajes a cada posible dato que aparezca en las categorías:

- Tipo.
- Subtipo.
- Propietario
- Verificación

In [17]:
duplicates_subset.loc[duplicates_subset['VERIFICADO'] == True, 'PUNTAJE_VERIFICADO'] = 6 

In [18]:
duplicates_subset.loc[duplicates_subset['VERIFICADO'] == False, 'PUNTAJE_VERIFICADO'] = 0

In [19]:
duplicates_subset.loc[duplicates_subset['PROPIETARIO_DATO'] == 'Propio', 'PUNTAJE_PROPIETARIO'] = 5

In [20]:
duplicates_subset.loc[duplicates_subset['PROPIETARIO_DATO'] == 'Ajeno_familiar', 'PUNTAJE_PROPIETARIO'] = 4

In [21]:
duplicates_subset.loc[duplicates_subset['PROPIETARIO_DATO'] == 'Ajeno_empleador', 'PUNTAJE_PROPIETARIO'] = 3

In [22]:
duplicates_subset.loc[duplicates_subset['PROPIETARIO_DATO'] == 'Ajeno', 'PUNTAJE_PROPIETARIO'] = 2

In [23]:
duplicates_subset.loc[duplicates_subset['SUBTIPO'] == 'PERSONAL', 'PUNTAJE_TIPO'] = 2

In [24]:
duplicates_subset.loc[duplicates_subset['SUBTIPO'] == 'LABORAL', 'PUNTAJE_TIPO'] = 0

In [25]:
duplicates_subset.loc[duplicates_subset['TIPO'] == 'EMAIL', 'PUNTAJE_SUBTIPO'] = 4

In [26]:
duplicates_subset.loc[duplicates_subset['TIPO'] == 'TEL.FIJO', 'PUNTAJE_SUBTIPO'] = 2

In [27]:
duplicates_subset.loc[duplicates_subset['TIPO'] == 'TEL.CELULAR', 'PUNTAJE_SUBTIPO'] = 4

## Suma de Puntajes

In [28]:
columns_to_sum = ['PUNTAJE_VERIFICADO','PUNTAJE_PROPIETARIO','PUNTAJE_TIPO','PUNTAJE_SUBTIPO']

In [29]:
duplicates_subset['PUNTAJE_TOTAL'] = duplicates_subset[columns_to_sum].sum(axis=1)

In [30]:
duplicates_subset.head()

,IDMOROSO,TIPO_DOC,NRO_DOC,CUIT_CUIL,TIPO,SUBTIPO,DATO,PROPIETARIO_DATO,PROVEEDOR_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,SCORE_DATO,PUNTAJE_VERIFICADO,PUNTAJE_PROPIETARIO,PUNTAJE_TIPO,PUNTAJE_SUBTIPO,PUNTAJE_TOTAL
0,BBVA12063032,DNI,12063032,NaN,TEL.FIJO,PERSONAL,2293423790,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4,6.0,5.0,2.0,2.0,15.0
1,BBVA11431130,DNI,11431130,NaN,TEL.FIJO,PERSONAL,2994487723,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4,6.0,5.0,2.0,2.0,15.0
2,BBVA22000834,DNI,22000834,NaN,TEL.FIJO,PERSONAL,4290892318,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4,6.0,5.0,2.0,2.0,15.0
3,BBVA22264120,DNI,22264120,NaN,TEL.FIJO,PERSONAL,3813073596,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4,6.0,5.0,2.0,2.0,15.0
4,BBVA16805508,DNI,16805508,NaN,TEL.FIJO,PERSONAL,3462431099,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4,6.0,5.0,2.0,2.0,15.0


In [31]:
duplicates_subset.shape

(14316, 19)

In [32]:
duplicates_subset[duplicates_subset['IDMOROSO'] == 'DNI 4558814       ']

,IDMOROSO,TIPO_DOC,NRO_DOC,CUIT_CUIL,TIPO,SUBTIPO,DATO,PROPIETARIO_DATO,PROVEEDOR_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,SCORE_DATO,PUNTAJE_VERIFICADO,PUNTAJE_PROPIETARIO,PUNTAJE_TIPO,PUNTAJE_SUBTIPO,PUNTAJE_TOTAL


## Agrupo

In [33]:
#duplicates_subset['ID'] = duplicates_subset['ID'].astype(str)

In [34]:
df_best = duplicates_subset.groupby(['IDMOROSO','DATO','PROVEEDOR_DATO'], as_index = False).max(numeric_only=True)

In [35]:
df_best

,IDMOROSO,DATO,PROVEEDOR_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,SCORE_DATO,PUNTAJE_VERIFICADO,PUNTAJE_PROPIETARIO,PUNTAJE_TIPO,PUNTAJE_SUBTIPO,PUNTAJE_TOTAL
0,BBVA10207955,2994770697,AT,True,NaN,NaN,4,6.0,5.0,2.0,2.0,15.0
1,BBVA10271332,2614523189,AT,True,NaN,NaN,4,6.0,5.0,2.0,2.0,15.0
2,BBVA10606208,47872609,AT,True,NaN,NaN,4,6.0,5.0,2.0,2.0,15.0
3,BBVA10962654,2944428196,AT,True,NaN,NaN,4,6.0,5.0,2.0,2.0,15.0
4,BBVA10964578,48376200,AT,True,NaN,NaN,4,6.0,5.0,2.0,2.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7153,RMS379977453132005,1539108765,AT,True,NaN,NaN,4,6.0,5.0,2.0,4.0,17.0
7154,RMS379977515062000,1524568940,AT,True,NaN,NaN,4,6.0,5.0,2.0,4.0,17.0
7155,RMS379977706022003,247415446380,AT,True,NaN,NaN,4,6.0,5.0,2.0,2.0,15.0
7156,RMS379977897042000,46058535,AT,True,NaN,NaN,4,6.0,5.0,2.0,2.0,15.0


# Join

In [36]:
duplicates_subset.head()

,IDMOROSO,TIPO_DOC,NRO_DOC,CUIT_CUIL,TIPO,SUBTIPO,DATO,PROPIETARIO_DATO,PROVEEDOR_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,SCORE_DATO,PUNTAJE_VERIFICADO,PUNTAJE_PROPIETARIO,PUNTAJE_TIPO,PUNTAJE_SUBTIPO,PUNTAJE_TOTAL
0,BBVA12063032,DNI,12063032,NaN,TEL.FIJO,PERSONAL,2293423790,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4,6.0,5.0,2.0,2.0,15.0
1,BBVA11431130,DNI,11431130,NaN,TEL.FIJO,PERSONAL,2994487723,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4,6.0,5.0,2.0,2.0,15.0
2,BBVA22000834,DNI,22000834,NaN,TEL.FIJO,PERSONAL,4290892318,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4,6.0,5.0,2.0,2.0,15.0
3,BBVA22264120,DNI,22264120,NaN,TEL.FIJO,PERSONAL,3813073596,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4,6.0,5.0,2.0,2.0,15.0
4,BBVA16805508,DNI,16805508,NaN,TEL.FIJO,PERSONAL,3462431099,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4,6.0,5.0,2.0,2.0,15.0


In [37]:
df_best.head()

,IDMOROSO,DATO,PROVEEDOR_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,SCORE_DATO,PUNTAJE_VERIFICADO,PUNTAJE_PROPIETARIO,PUNTAJE_TIPO,PUNTAJE_SUBTIPO,PUNTAJE_TOTAL
0,BBVA10207955,2994770697,AT,True,NaN,NaN,4,6.0,5.0,2.0,2.0,15.0
1,BBVA10271332,2614523189,AT,True,NaN,NaN,4,6.0,5.0,2.0,2.0,15.0
2,BBVA10606208,47872609,AT,True,NaN,NaN,4,6.0,5.0,2.0,2.0,15.0
3,BBVA10962654,2944428196,AT,True,NaN,NaN,4,6.0,5.0,2.0,2.0,15.0
4,BBVA10964578,48376200,AT,True,NaN,NaN,4,6.0,5.0,2.0,2.0,15.0


In [38]:
df_joined = pd.merge(duplicates_subset,df_best,on=['IDMOROSO','DATO','PROVEEDOR_DATO','PUNTAJE_TOTAL'])

In [39]:
df_joined.shape

(7158, 27)

In [40]:
df_joined.head(10)

,IDMOROSO,TIPO_DOC,NRO_DOC,CUIT_CUIL,TIPO,SUBTIPO,DATO,PROPIETARIO_DATO,PROVEEDOR_DATO,FECHA_INGRESO_DATO,...,PUNTAJE_SUBTIPO_x,PUNTAJE_TOTAL,VERIFICADO_y,FECHA_VERIFICACION_y,OPERADOR_VERIFICACION_y,SCORE_DATO_y,PUNTAJE_VERIFICADO_y,PUNTAJE_PROPIETARIO_y,PUNTAJE_TIPO_y,PUNTAJE_SUBTIPO_y
0,BBVA12063032,DNI,12063032,NaN,TEL.FIJO,PERSONAL,2293423790,Propio,AT,2021-01-14 16:03:30.252685,...,2.0,15.0,True,NaN,NaN,4,6.0,5.0,2.0,2.0
1,BBVA11431130,DNI,11431130,NaN,TEL.FIJO,PERSONAL,2994487723,Propio,AT,2021-01-14 16:03:30.252685,...,2.0,15.0,True,NaN,NaN,4,6.0,5.0,2.0,2.0
2,BBVA22000834,DNI,22000834,NaN,TEL.FIJO,PERSONAL,4290892318,Propio,AT,2021-01-14 16:03:30.252685,...,2.0,15.0,True,NaN,NaN,4,6.0,5.0,2.0,2.0
3,BBVA22264120,DNI,22264120,NaN,TEL.FIJO,PERSONAL,3813073596,Propio,AT,2021-01-14 16:03:30.252685,...,2.0,15.0,True,NaN,NaN,4,6.0,5.0,2.0,2.0
4,BBVA16805508,DNI,16805508,NaN,TEL.FIJO,PERSONAL,3462431099,Propio,AT,2021-01-14 16:03:30.252685,...,2.0,15.0,True,NaN,NaN,4,6.0,5.0,2.0,2.0
5,BBVA29730954,DNI,29730954,NaN,TEL.FIJO,PERSONAL,3772421801,Propio,AT,2021-01-14 16:03:30.252685,...,2.0,15.0,True,NaN,NaN,4,6.0,5.0,2.0,2.0
6,BSOL10017903,CUIT,BSOL10017903,NaN,TEL.FIJO,PERSONAL,3856990620,Propio,AT,2021-01-14 16:03:30.252685,...,2.0,15.0,True,NaN,NaN,4,6.0,5.0,2.0,2.0
7,BBVA6135477,DNI,6135477,NaN,TEL.CELULAR,PERSONAL,1544119818,Propio,AT,2021-01-14 16:03:30.252685,...,4.0,17.0,True,NaN,NaN,4,6.0,5.0,2.0,4.0
8,BSOL10295751,CUIT,BSOL10295751,NaN,TEL.FIJO,PERSONAL,382515542989,Propio,AT,2021-01-14 16:03:30.252685,...,2.0,15.0,True,NaN,NaN,4,6.0,5.0,2.0,2.0
9,BBVA21656871,DNI,21656871,NaN,TEL.FIJO,PERSONAL,43212500,Propio,AT,2021-01-14 16:03:30.252685,...,2.0,15.0,True,NaN,NaN,4,6.0,5.0,2.0,2.0


# Duplicados con diferentes fechas de ingreso

In [41]:
duplicates_subset[duplicates_subset['IDMOROSO'] == 'COL 14961623      ']

,IDMOROSO,TIPO_DOC,NRO_DOC,CUIT_CUIL,TIPO,SUBTIPO,DATO,PROPIETARIO_DATO,PROVEEDOR_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,SCORE_DATO,PUNTAJE_VERIFICADO,PUNTAJE_PROPIETARIO,PUNTAJE_TIPO,PUNTAJE_SUBTIPO,PUNTAJE_TOTAL


In [42]:
duplicates_fecha = df_joined[df_joined.duplicated(subset=['IDMOROSO','DATO','PROVEEDOR_DATO'],keep=False)]

In [43]:
duplicates_fecha = duplicates_fecha.sort_values('FECHA_INGRESO_DATO').drop_duplicates('IDMOROSO',keep='first')

# Append

In [44]:
df_joined = df_joined[['IDMOROSO','TIPO_DOC','NRO_DOC','CUIT_CUIL','TIPO','SUBTIPO','DATO','PROPIETARIO_DATO'
                               ,'PROVEEDOR_DATO','FECHA_INGRESO_DATO','VERIFICADO_x','FECHA_VERIFICACION_x'
                               ,'OPERADOR_VERIFICACION_x','SCORE_DATO_x']]

In [45]:
df_joined.head()

,IDMOROSO,TIPO_DOC,NRO_DOC,CUIT_CUIL,TIPO,SUBTIPO,DATO,PROPIETARIO_DATO,PROVEEDOR_DATO,FECHA_INGRESO_DATO,VERIFICADO_x,FECHA_VERIFICACION_x,OPERADOR_VERIFICACION_x,SCORE_DATO_x
0,BBVA12063032,DNI,12063032,NaN,TEL.FIJO,PERSONAL,2293423790,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4
1,BBVA11431130,DNI,11431130,NaN,TEL.FIJO,PERSONAL,2994487723,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4
2,BBVA22000834,DNI,22000834,NaN,TEL.FIJO,PERSONAL,4290892318,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4
3,BBVA22264120,DNI,22264120,NaN,TEL.FIJO,PERSONAL,3813073596,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4
4,BBVA16805508,DNI,16805508,NaN,TEL.FIJO,PERSONAL,3462431099,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4


In [46]:
df_joined.shape

(7158, 14)

In [47]:
df_joined = df_joined.rename(columns={"VERIFICADO_x": "VERIFICADO", "SCORE_DATO_x": "SCORE_DATO"
                                     ,'NRO_DOC_x': 'NRO_DOC','CUIT_CUIL_x':'CUIT_CUIL'
                                     ,'FECHA_VERIFICACION_x':'FECHA_VERIFICACION'})

In [48]:
df_joined = df_joined[~df_joined.duplicated(subset=['IDMOROSO','DATO','PROVEEDOR_DATO'],keep=False)]

In [49]:
df_joined.shape

(7158, 14)

In [50]:
duplicates_fecha = duplicates_fecha[['IDMOROSO','TIPO_DOC','NRO_DOC','CUIT_CUIL','TIPO','SUBTIPO','DATO','PROPIETARIO_DATO'
                               ,'PROVEEDOR_DATO','FECHA_INGRESO_DATO','VERIFICADO_x','FECHA_VERIFICACION_x'
                               ,'OPERADOR_VERIFICACION_x','SCORE_DATO_x']]

In [51]:
duplicates_fecha.shape

(0, 14)

In [52]:
duplicates_fecha = duplicates_fecha.rename(columns={"VERIFICADO_x": "VERIFICADO", "SCORE_DATO_x": "SCORE_DATO"})

Dataframe inicial sin los duplicados

In [53]:
df.shape

(14316, 14)

In [54]:
non_duplicates = df[~df.duplicated(subset=['IDMOROSO','DATO','PROVEEDOR_DATO'],keep=False)]

In [55]:
non_duplicates.shape

(0, 14)

Unimos los 3 dataframes y revisamos que no haya duplicados en el mismo subset IDMOROSO-DATO-PROVEEDOR

In [56]:
non_duplicates = non_duplicates.dropna(subset=['DATO'])

In [57]:
non_duplicates [non_duplicates['NRO_DOC'].isnull()]

,IDMOROSO,TIPO_DOC,NRO_DOC,CUIT_CUIL,TIPO,SUBTIPO,DATO,PROPIETARIO_DATO,PROVEEDOR_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,SCORE_DATO


In [58]:
non_duplicates.head()

,IDMOROSO,TIPO_DOC,NRO_DOC,CUIT_CUIL,TIPO,SUBTIPO,DATO,PROPIETARIO_DATO,PROVEEDOR_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,SCORE_DATO


In [59]:
#df_export = pd.concat([non_duplicates,df_joined,duplicates_fecha])
df_export = pd.concat([non_duplicates,df_joined])

In [60]:
df_export.head()

,IDMOROSO,TIPO_DOC,NRO_DOC,CUIT_CUIL,TIPO,SUBTIPO,DATO,PROPIETARIO_DATO,PROVEEDOR_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,SCORE_DATO,OPERADOR_VERIFICACION_x
0,BBVA12063032,DNI,12063032,NaN,TEL.FIJO,PERSONAL,2293423790,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4,NaN
1,BBVA11431130,DNI,11431130,NaN,TEL.FIJO,PERSONAL,2994487723,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4,NaN
2,BBVA22000834,DNI,22000834,NaN,TEL.FIJO,PERSONAL,4290892318,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4,NaN
3,BBVA22264120,DNI,22264120,NaN,TEL.FIJO,PERSONAL,3813073596,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4,NaN
4,BBVA16805508,DNI,16805508,NaN,TEL.FIJO,PERSONAL,3462431099,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4,NaN


In [61]:
df_export.shape

(7158, 15)

# QA Duplicados

In [62]:
qa_duplicados = df_export[df_export.duplicated(subset=['IDMOROSO','DATO','PROVEEDOR_DATO'],keep=False)]

In [63]:
qa_duplicados

,IDMOROSO,TIPO_DOC,NRO_DOC,CUIT_CUIL,TIPO,SUBTIPO,DATO,PROPIETARIO_DATO,PROVEEDOR_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,SCORE_DATO,OPERADOR_VERIFICACION_x


In [64]:
df_export.head()

,IDMOROSO,TIPO_DOC,NRO_DOC,CUIT_CUIL,TIPO,SUBTIPO,DATO,PROPIETARIO_DATO,PROVEEDOR_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,SCORE_DATO,OPERADOR_VERIFICACION_x
0,BBVA12063032,DNI,12063032,NaN,TEL.FIJO,PERSONAL,2293423790,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4,NaN
1,BBVA11431130,DNI,11431130,NaN,TEL.FIJO,PERSONAL,2994487723,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4,NaN
2,BBVA22000834,DNI,22000834,NaN,TEL.FIJO,PERSONAL,4290892318,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4,NaN
3,BBVA22264120,DNI,22264120,NaN,TEL.FIJO,PERSONAL,3813073596,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4,NaN
4,BBVA16805508,DNI,16805508,NaN,TEL.FIJO,PERSONAL,3462431099,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4,NaN


In [65]:
df_export['VERIFICADO'].value_counts()

True    7158
Name: VERIFICADO, dtype: int64

In [66]:
non_duplicates['VERIFICADO'].value_counts()

Series([], Name: VERIFICADO, dtype: int64)

In [67]:
df_joined['VERIFICADO'].value_counts()

True    7158
Name: VERIFICADO, dtype: int64

In [68]:
duplicates_fecha['VERIFICADO'].value_counts()

Series([], Name: VERIFICADO, dtype: int64)

In [69]:
df_export[df_export['IDMOROSO'] == 'DNI 4558814       ']

,IDMOROSO,TIPO_DOC,NRO_DOC,CUIT_CUIL,TIPO,SUBTIPO,DATO,PROPIETARIO_DATO,PROVEEDOR_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,SCORE_DATO,OPERADOR_VERIFICACION_x


In [80]:
df_export[df_export['IDMOROSO'] == 'DNIF31872573      ']

,IDMOROSO,TIPO_DOC,NRO_DOC,CUIT_CUIL,TIPO,SUBTIPO,DATO,PROPIETARIO_DATO,PROVEEDOR_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,SCORE_DATO
2178,DNIF31872573,CUIT,F31872573,NaN,TEL.FIJO,PERSONAL,362154724547,Propio,AT,2021-01-14 16:03:30.252685,True,NaN,NaN,4


# Export a DAM_CONTACTABILIDAD

In [71]:
df_export = df_export[['IDMOROSO', 'TIPO_DOC','NRO_DOC','CUIT_CUIL','TIPO','SUBTIPO','DATO','PROPIETARIO_DATO','PROVEEDOR_DATO','FECHA_INGRESO_DATO','VERIFICADO','FECHA_VERIFICACION','OPERADOR_VERIFICACION','SCORE_DATO']]

In [72]:
#df_export.to_csv('D:/MOPC/Enriquecimiento de Datos/Data Cleaning/Active/Exports/DAM_CONTACTABILIDAD/telefonos_non_dups.csv',index=False)